In [ ]:
#разбиваем всю БД каждые 10000 строк на разделы lim 
import pymysql.cursors
import pandas as pd
import numpy as np

import re
import emoji

import pymorphy2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))

from google.colab import drive
drive.mount('/gdrive')

# препроцессинг

def preprocess_text(text):
    def replace_three_or_more(text):
        pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
        return pattern.sub(r"\1\1", text)

    def emo_to_text(text):  
        emoticon = {'улыбка шутка' : ['\:\)', '\:\-\)', '\:\=\)'],
                    'грусть разочарование' : ['\:\(', '\:\-\(', '\:\=\('],
                    'открытая радость' : ['\=\)', '\=\-\)'],
                    'усмешка хихиканье' : ['\:>', '\:\->', '\:\=>'],
                    'улыбка' : ['\:\]', '\:\-\]', '\:\=\]'],
                    'смех' : ['\:D', '\:\-D', '\:\=D'],
                    'сильный смех' : ['\:DD', '\:\-DD', '\:\=DD'],
                    'сарказм' : ['\:\}', '\:\-\}', '\:\=\}'],
                    'подмигивание заигрывание' : ['\;\)', '\;\-\)', '\;\=\)'],
                    'молчание' : ['\:Х', '\:\-Х', '\:\=Х'],
                    'удивленное разочарование' : ['8\(', '8\-\(', '8\=\('],
                    'изумление' : ['B\-о', 'B\=о'],
                    'радостное удивление' : ['\%\)'],
                    'зевота' : ['\|\-o', '\|\=o'],
                    'смущение' : ['\:S', '\:-S', '\:\=S'],
                    'равнодушие скука' : ['\:\|', '\:\-\|', '\:\=\|'],
                    'недоверие сомнение' : ['\:\/', '\:\-\/', '\:\=\/'],
                    'дразнящее показывание языка' : ['\:ь', '\:\-ь', '\:\=ь'],
                    'поцелуй' : ['\:\*', '\:\^\*', '\:\-Ф'],
                    'шалость дурачество' : ['\:0\)'],
                    'кривая ухмылка' : ['\:7', '\:\-7']}

        for key, value in emoticon.items():
          text = re.sub(r"|".join(value), ' ' + key, text)
        return text

    try:
      text = emo_to_text(text)
      text = emoji.demojize(text, delimiters=("", "")) #преобразует эмодзи-символы в слова
      text = text.lower().replace("ё", "е")
      text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
      text = re.sub('@[^\s]+', 'USER', text)
      text = re.sub('[^a-zA-Zа-яА-Я]+', ' ', text) #удаляем цифры и спецсимволы
      text = replace_three_or_more(text) #удаляет дублирование букв, если число букв более 3
      text = re.sub(' +', '|', text)
      return text.strip()
    except:
      print(f'Слово {text} не прошло препроцессинг')
      return ''

# Connect to the database
def get_connection():
    connection = pymysql.connect(host='database-3.chim8btj05zt.us-east-1.rds.amazonaws.com',
                                 user='admin',
                                 password='01035007',
                                 db='test',
                                 charset='utf8mb4',
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection


connection = get_connection()

morph = pymorphy2.MorphAnalyzer() #по умолчанию русский язык
stemmer = SnowballStemmer("english")

try:
    with connection.cursor() as cursor:

        #cursor.execute("DROP TABLE IF EXISTS linis_texts")
        #connection.commit()
        cursor.execute("""CREATE TABLE IF NOT EXISTS linis_texts(
              id bigint(32) NOT NULL PRIMARY KEY,
              ttext_stem varchar(256) DEFAULT NULL,
              ttext_prep varchar(256) DEFAULT NULL,
              ttext varchar(256) DEFAULT NULL,
              ttype int(10) DEFAULT '0') DEFAULT CHARSET=utf8mb4""")
        connection.commit()
        
        data1 = pd.read_csv("/gdrive/My Drive/Colab Notebooks/doc_test/text_2015_.csv", sep = ';')
        data1 = data1.iloc[:, :2]
        data1.columns = ['ttext', 'ttype']
        print(data1.head(5))
        print(f"В датафрейм загружены прецеденты в количестве {data1.shape}")
        
        data2 = pd.read_csv("/gdrive/My Drive/Colab Notebooks/doc_test/text_2016_.csv", sep = ';')
        data2.columns = ['ttext', 'ttype']
        print(data2.head(5))
        print(f"В датафрейм загружены прецеденты в количестве {data2.shape}")
        
        raw_data = pd.concat([data1, data2], ignore_index = True)
        raw_data = raw_data.drop_duplicates()
        raw_data = raw_data.sample(frac=1).reset_index(drop=True)
        print(raw_data.shape)

        #raw_data = raw_data.head(100)
        
        #препроцессинг
        if raw_data.shape[0] > 0:
          print(f"Первая строка: {raw_data.head(1)}")
          raw_data['ttext_prep'] = raw_data['ttext'].apply(preprocess_text)      

        #стемминг
        ttext_stem = list()
        for row in raw_data.itertuples():
            words = row.ttext_prep.split('|')
            nf_words = list()
            for word in words:
              try:
                #word = word.encode('utf8mb4')
                ru_word = re.sub('[^а-яА-Я]+', ' ', word)
                en_word = re.sub('[^a-zA-Z]+', ' ', word)
                if ru_word == word and word != '': # кириллица
                  parse_word = morph.parse(word)[0]
                  nf_word = parse_word.normal_form
                  nf_words.append(nf_word.strip())
                elif en_word == word and word != '': # латиница
                  nf_word = stemmer.stem(word)
                  nf_words.append(nf_word.strip())
                else:
                  pass #смешанные слова не анализируются
              except:
                print(f'Слово не преобразовано') 
                continue
            nf_words = [w for w in nf_words if not w in stops]
            nf_words = '|'.join(nf_words) #ttext_stem
            ttext_stem.append(nf_words)

        if ttext_stem != None:
            ttext_stem = pd.Series(ttext_stem) 
            raw_data['ttext_stem'] = ttext_stem  
            insert = """INSERT INTO linis_texts VALUES (%s, %s, %s, %s, %s)"""
            for row in raw_data.itertuples():
                #print(row.Index)
                cursor.execute(insert, (row.Index, row.ttext_stem, row.ttext_prep, row.ttext, row.ttype))
                connection.commit()
   
finally:
    connection.close()



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
                                               ttext ttype
0  Когда возникнут трудности, они тебе не помогут...     0
1  Кривая национализация это политический компром...    -1
2  Такой вид биологического оружия не действует н...    -2
3  В Эль-Кусейре /к западу от Хомса/ сирийские по...     0
4  В том числе, ограничениями на продажу спиртных...     0
В датафрейм загружены прецеденты в количестве (32436, 2)
                                               ttext ttype
0  Украина это часть Руси искусственно отделенная...    -1
1  Как можно говорить об относительно небольшой к...    -1
2  1.2014.  а что они со своими поляками сделали?...     0
3  у а фильмы... Зрители любят диковинное.   у ме...     0
4  Государство не может сейчас платить больше и м...    -1
В датафрейм загружены прецеденты в количестве (26872, 2)
(45631, 2)
Первая строка:                                      

NameError: ignored

In [12]:
print(raw_data.shape)
try:
    connection = get_connection()
    with connection.cursor() as cursor:
        for row in raw_data.itertuples():
            try:
                #print(row.Index)
                cursor.execute(insert, (row.Index, row.ttext_stem, row.ttext_prep, row.ttext, row.ttype))
                connection.commit()
            except Exception as e:
                print(f"Error {e}")
                continue
finally:
    connection.close()        

(45631, 4)
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
Error name 'ProgrammingError' is not defined
